# YOLOv8 Bone Fracture Detection

## Downloading Bone Fracture Dataset

Your Kaggle API token can be downloaded from https://www.kaggle.com/settings/account

Retrieve the `Kaggle.json` file you downloaded and **make sure it is in the runtime directory** of the Colab notebook. I have automated this by storing the file inside my google drive and using ```drive.mount('/content/drive')```. If you are going to use this method, ensure you change the path to your file. Otherwise you can manually update the file and omitt the relevant code.

After this, we modify the values in the .yaml data file to reflect the new directory.

In [ ]:
!pip install -q opendatasets

from google.colab import drive
import os, sys
import opendatasets as od
import yaml

kaggle_token_path = '/content/drive/MyDrive/AISI-Project/kaggle.json'# REPLACE WITH OWN PATH

# Retrieving Kaggle API token from my google drive, change this if you plan on retrieving from elsewhere.
drive.mount('/content/drive')
!cp "$kaggle_token_path" . # Copy the kaggle.json file to the current working directory
drive.flush_and_unmount()

# Downloading dataset
od.download("https://www.kaggle.com/datasets/pkdarabi/bone-fracture-detection-computer-vision-project", data_dir='./datasets')

# Modifying YAML file to work with YOLOv8
yaml_path = '/content/datasets/bone-fracture-detection-computer-vision-project/data.yaml'

with open(yaml_path, 'r') as file:
    data = yaml.safe_load(file)

data['train'] = '/content/datasets/bone-fracture-detection-computer-vision-project/train/images'
data['val'] = '/content/datasets/bone-fracture-detection-computer-vision-project/valid/images'
data['test'] = '/content/datasets/bone-fracture-detection-computer-vision-project/test/images'

with open(yaml_path, 'w') as file:
    yaml.dump(data, file)

print("YAML file updated.")

Mounted at /content/drive


100%|██████████| 41.6M/41.6M [00:03<00:00, 13.6MB/s]



YAML file updated.


## Installing YOLOv8

In [ ]:
# Remove -q argument if things are going wrong, it just hides the pip output
!pip install -q git+https://github.com/ultralytics/ultralytics.git@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

## Defining and Training YOLOv8 Model

In [ ]:
from ultralytics import YOLO

yaml_path = "./datasets/bone-fracture-detection-computer-vision-project/data.yaml"

# Custom Parameters
# Parameter | Value | Description:

# save	      True	  save train checkpoints and predict results
# batch	       -1    	number of images per batch (-1 for AutoBatch)
# patience	   50	    epochs to wait for no observable improvement for early stopping of training
# plots	      False	  save plots and images during train/val

model = YOLO('yolov8n.pt')
results = model.train(data=yaml_path, epochs=100, imgsz=350, batch=-1, plots=True, save=True, patience = 10)
#results = model.tune(data=yaml_path, epochs=30, imgsz=512, batch=-1, iterations=300, save=False)

100%|██████████| 6.23M/6.23M [00:00<00:00, 22.9MB/s]


Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./datasets/bone-fracture-detection-computer-vision-project/data.yaml, epochs=100, time=None, patience=10, batch=-1, imgsz=350, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fals

100%|██████████| 755k/755k [00:00<00:00, 4.79MB/s]


Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/datasets/bone-fracture-detection-computer-vision-project/train/labels... 3631 images, 1827 backgrounds, 0 corrupt: 100%|██████████| 3631/3631 [00:01<00:00, 1964.90it/s]

...

     39/100      12.3G      1.654      1.362      1.181        302        352: 100%|██████████| 12/12 [00:25<00:00,  2.14s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.10s/it]

                   all        348        204      0.405      0.262      0.229      0.088
Stopping training early as no improvement observed in last 10 epochs. Best results observed at epoch 29, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



39 epochs completed in 0.321 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.62s/it]


                   all        348        204      0.338      0.299      0.259      0.105
        elbow positive        348         29      0.147      0.103      0.081     0.0295
      fingers positive        348         48       0.25      0.146      0.157     0.0465
      forearm fracture        348         43      0.509      0.302      0.272      0.104
               humerus        348         36      0.466      0.583      0.576      0.212
     shoulder fracture        348         20      0.339       0.55      0.395      0.221
        wrist positive        348         28      0.316      0.107     0.0728     0.0183
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train


## Model Prediction

In [ ]:
from ultralytics import YOLO

predict = model('/content/datasets/bone-fracture-detection-computer-vision-project/test/images/image2_199_png.rf.111ecddb2bdc3542d7f953385d1bb03b.jpg', visualize=True, save=True, save_txt=True)


Saving runs/detect/train2/image2_199_png.rf.111ecddb2bdc3542d7f953385d1bb03b/stage0_Conv_features.png... (16/16)
Saving runs/detect/train2/image2_199_png.rf.111ecddb2bdc3542d7f953385d1bb03b/stage1_Conv_features.png... (32/32)
Saving runs/detect/train2/image2_199_png.rf.111ecddb2bdc3542d7f953385d1bb03b/stage2_C2f_features.png... (32/32)
Saving runs/detect/train2/image2_199_png.rf.111ecddb2bdc3542d7f953385d1bb03b/stage3_Conv_features.png... (32/64)
Saving runs/detect/train2/image2_199_png.rf.111ecddb2bdc3542d7f953385d1bb03b/stage4_C2f_features.png... (32/64)
Saving runs/detect/train2/image2_199_png.rf.111ecddb2bdc3542d7f953385d1bb03b/stage5_Conv_features.png... (32/128)
Saving runs/detect/train2/image2_199_png.rf.111ecddb2bdc3542d7f953385d1bb03b/stage6_C2f_features.png... (32/128)
Saving runs/detect/train2/image2_199_png.rf.111ecddb2bdc3542d7f953385d1bb03b/stage7_Conv_features.png... (32/256)
Saving runs/detect/train2/image2_199_png.rf.111ecddb2bdc3542d7f953385d1bb03b/stage8_C2f_feature

## Exporting Model

In [ ]:
model.export(format='onnx')

Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 352, 352) BCHW and output shape(s) (1, 11, 2541) (5.9 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 46.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 8.7s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 9.4s, saved as 'runs/detect/train/weights/best.onnx' (11.6 MB)

Export complete (10.8s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.onnx imgsz=352  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.onnx imgsz=352 data=./datasets/bone-fracture-detection-com

'runs/detect/train/weights/best.onnx'

## Downloading Results

In [ ]:
import shutil
from google.colab import files

TARGET_FOLDER = "/content/runs/detect/train"
OUTPUT_ZIP = "/content/output.zip"

# Create a zip file
shutil.make_archive(OUTPUT_ZIP.replace('.zip', ''), 'zip', TARGET_FOLDER)

# Download the file
files.download(OUTPUT_ZIP)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>